Intentions:\
I want to perform a classification task to get the direction of the S&P500 for the next day.
As input features i take the returns of the last day from some of the best capitalized stocks inside the S&P500.
The raw data will be fetched via the Yahoo finance API.

The classification model will be a simply Support Vector Classifier, the data will be very noisy
so more complex classfiers are not recommended.

For evaluation i will be perform a TimeSeriesCV with a moving window of one year (250d). The input data ranges from 2015-2022 for the training data
analyzed with the TimeSeriesCV and also the validation data as a holdout set will be the year 2023.

The Evaluation Report contains classic metrics for classification task like precision and accurcay, but also decision
making data for a trading system like the total returns or the return per trade. Evaluation Reports will be generated for training and validation set.

The data transformation (e.g. from the raw OHLC data to returns) will be done through a pipeline, additionally i set up dynamic variable naming so i am able to quickly generate different models for comparision purposes.

Since i am using data with likely no significant impact on the model, i will reduce the data in a seperate model via a simply kbest approach and compare it with the Baseline mode.

# 1. Fetching Data

In [1]:

import numpy as np
import pandas as pd
from itertools import groupby
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif
import warnings
warnings.filterwarnings("ignore")
df_Ticker = pd.DataFrame()
ticker_list = ["^SPX", "AAPL","MSFT","AMZN","NVDA","META","TSLA","V","UNH","JPM",
               "JNJ","LLY","WMT","XOM","PG","AVGO","MA","HD","ORCL",
               "CVX","MRK","KO","ABBV","PEP","BAC","COST","ADBE","CRM","CSCO",
               "TMO","MCD","ACN","PFE","NFLX","DHR","ABT","LIN","CMCSA",
               "AMD","NKE","TMUS","DIS","UPS","TXN","PM","MS","CAT","NEE",
               "INTC","QCOM","UNP","VZ","COP","BA","INTU","LOW","IBM","BMY",
               "HON","DE","SPGI","BX","RTX","AMAT","AXP","GE","SCHW","SBUX",
               "GS","NOW","MDT","LMT","ELV","ISRG","BLK","BKNG","SYK","T",
               "MDLZ","ADP","TJX","CVS","ADI","GILD","MMC","VRTX","LRCX",
               "C","CI","ETN","CB","ZTS","WFC","SLB","REGN"]
for i in ticker_list:
    df_single=(yf.download([i], start="2015-01-01", end="2023-12-31"))
    df_single['symbol'] = i
    df_Ticker=pd.concat([df_Ticker, df_single], ignore_index=False)
################## delete NaN
df_Ticker = df_Ticker.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

# Functions


## Evaluation Report Function

In [2]:
##### Evaluation Report
def cf_matrix_generation(Prediction_Value,True_Value):
    if Prediction_Value > 0 and True_Value > 0:
        return "TP"
    elif Prediction_Value > 0 and True_Value <= 0:
        return "FP"
    elif Prediction_Value <= 0 and True_Value <= 0:
        return "TN"
    elif Prediction_Value <= 0 and True_Value > 0:
        return "FN"
  
def Evaluation_generation(model_data, Prediction_Value, True_Value, numeric_value):
    model_data['cf_matrix_data'] = model_data.apply(lambda row: cf_matrix_generation(row[Prediction_Value], row[True_Value]), axis=1)
    TP_data = model_data.loc[model_data['cf_matrix_data'] == 'TP']
    FP_data = model_data.loc[model_data['cf_matrix_data'] == 'FP']
    TN_data = model_data.loc[model_data['cf_matrix_data'] == 'TN']
    FN_data = model_data.loc[model_data['cf_matrix_data'] == 'FN']
    TP = len(TP_data)
    FP = len(FP_data)
    TN = len(TN_data)
    FN = len(FN_data)
    if TP == 0:
        TP = np.nan
    if FP == 0:
        FP = np.nan
    if TN == 0:
        TN = np.nan
    if FP == 0:
        FP = np.nan
    #####Overall
    #Total Net Profit
    Total_Trades = TP + FP + TN + FN
    #Max_Drawdown
    Percent_in_the_Market = (TP + FP + TN + FN)/model_data.index.size
    #####Trades Long
    Total_Winners_Long = TP
    Precision = TP/(TP + FP)
    Long_Ratio = (TP + FN)/(TN + FN + TP + FP)
    Return_pct_Long = TP_data[numeric_value].sum() + FP_data[numeric_value].sum()
    Loss_pct_Long = -FP_data[numeric_value].sum()
    Profit_Factor_Long = Return_pct_Long/Loss_pct_Long
    Average_Win_Long = TP_data[numeric_value].sum()/TP
    Largest_Win_Long = TP_data[numeric_value].max()    
    Total_Losers_Long = FP
    Average_Loss_Long = FP_data[numeric_value].sum()/FP
    Largest_Loss_Long = FP_data[numeric_value].min()
    Average_Trade_Long = Return_pct_Long/(TP+FP)
    #####Trades Short
    Total_Winners_Short = TN
    NPV = TN/(TN + FN)
    Short_Ratio = (TN + FP)/(TN + FN + TP + FP)
    Return_pct_Short = -TN_data[numeric_value].sum() - FN_data[numeric_value].sum()
    Loss_pct_Short = FN_data[numeric_value].sum()
    Profit_Factor_Short = Return_pct_Short/Loss_pct_Short
    #Gross Profit
    Average_Win_Short = -(TN_data[numeric_value].sum())/TN
    Largest_Win_Short = -TN_data[numeric_value].min()       
    Total_Losers_Short = FN
    Average_Loss_Short = -FN_data[numeric_value].sum()/FN
    Largest_Loss_Short = -FN_data[numeric_value].max()
    Average_Trade_Short = Return_pct_Short/(TN+FN)   
#####Overall
    Total_Winners = TP + TN
    Accuracy = (TP + TN)/(TP + TN + FN + FP)
    Return_pct_lin = Return_pct_Long + Return_pct_Short    
    Total_Losers = FP + FN  
    Average_Trade = Return_pct_lin/Total_Trades
    Trade_Profit_Factor = (Return_pct_Long + Return_pct_Short)/(Loss_pct_Long + Loss_pct_Short)
    data=[['Overview','Overview'],['Accuracy',Accuracy],['Precision',Precision],['NPV',NPV],['Return pct lin',Return_pct_lin],
    ['Trade Profit Factor', Trade_Profit_Factor],['Average_Trade',Average_Trade],['Average Trade Long',Average_Trade_Long],
    ['Average Trade Short',Average_Trade_Short],
    ['Total Trades',Total_Trades],['Percent in the Market',Percent_in_the_Market],['Long Trades','Long Trades'],
    ['Total Winners Long' ,Total_Winners_Long],['Precision',Precision],['Long Ratio',Long_Ratio],['Return pc long',Return_pct_Long],
    ['Profit Factor Long',Profit_Factor_Long],['Average Trade Long',Average_Trade_Long],['Avergage Win Long',Average_Win_Long],
    ['Average Loss Long',Average_Loss_Long],
    ['Largest Win Long',Largest_Win_Long],['Total Losers Long',Total_Losers_Long],['Largest Loss Long',Largest_Loss_Long ],
    ['Short Trades','Short Trades'],['Total Winners Short',Total_Winners_Short],['Negative Predictive Value',NPV],
    ['Short Ratio',Short_Ratio],['Return pct Short',Return_pct_Short],['Profit Factor Short',Profit_Factor_Short],
    ['Average Trade Short',Average_Trade_Short],['Average Win Short',Average_Win_Short],
    ['Average Loss Short',Average_Loss_Short],['Largest Win Short',Largest_Win_Short],
    ['Total Losers Short',Total_Losers_Short],['Largest Loss Short',Largest_Loss_Short],
    ['Total Winners',Total_Winners],['General','General'],['Accuracy',Accuracy],['Total Losers',Total_Losers]]
    Evaluation = pd.DataFrame(data,columns=['Metric','Value'])
    return(Evaluation)


## Feature Generation and Pipeline Functions

In [3]:

#####Get a Transformed Column e.g Returns
def gettransformedOHLC(Ticker, dividend_column='Close', divisor_column = 'Close', dividend_shift = 0, divisor_shift = 1,  return_column='return_lag'):
    transformed_values = []
    for ticker in ticker_list:
        df = Ticker[Ticker['symbol'] == ticker]
        transformed_single_value = (df[dividend_column].shift(dividend_shift) / df[divisor_column].shift(divisor_shift) - 1) * 100
        transformed_values.append(transformed_single_value)
    Ticker[return_column] = pd.concat(transformed_values)
    return Ticker
##### Get a Transformed DF, e.g. Returns from all symbols
def gettransformedOHLC_df(Ticker, dividend_column='Close', divisor_column = 'Close', dividend_shift = 0, divisor_shift = 1, column_suffix='_return'):
    transformed_values = []
    transformed_values = {}
    for ticker in ticker_list:
        df = Ticker[Ticker['symbol'] == ticker]
        transformed_single_value = (df[dividend_column].shift(dividend_shift)/df[divisor_column].shift(divisor_shift)-1)*100
        transformed_values[f'{ticker}{column_suffix}'] = transformed_single_value
    transformed_values_df = pd.DataFrame(transformed_values)
    Ticker = pd.merge(Ticker, transformed_values_df, on="Date", how='left')
    return Ticker
##### Get Sum of column, e.g. Returns
def getsum_column(Ticker, column_to_sum='Returns', column_name_new = 'sumreturns'):
    Ticker[column_name_new] = Ticker.groupby('Date')[column_to_sum].transform('sum')
    return Ticker
#####Copy Data
def create_copy_data(data):
    data_copy = data.copy()
    return data_copy
#####Categorize Column
def simpleCat(x):
    if x > 0:
        return int(1)
    if x < 0:
        return int(0)
def apply_simpleCat(Ticker, column_name):
    Ticker[f'{column_name}Cat'] = [simpleCat(x) for x in Ticker[column_name].values]
    return Ticker
#####Filter Dataframe
def filter_by_symbol(Ticker, symbol):
    return Ticker[Ticker['symbol'] == symbol]
######Delete nan
def delete_nan(Ticker):
    Ticker = Ticker.dropna()
    return Ticker
#####Delete OHLC
def delete_OHLC(Ticker):
    del Ticker['Open']
    del Ticker['High']
    del Ticker['Low']
    del Ticker['Close']
    del Ticker['Volume']
    del Ticker['Adj Close']
    del Ticker['symbol']
    return Ticker
#####Delete Specific Column
def delete_spec_col(Ticker, column_name):
    del Ticker[column_name]
    return Ticker
#####Train/Val Split
def split_data_train_val(df, **kwargs):
    df_1 = df[(df.index > kwargs['start_date_train']) & (df.index < kwargs['end_date_train'])]
    df_2 = df[(df.index > kwargs['start_date_val']) & (df.index < kwargs['end_date_val'])]
    return df_1, df_2
##### Prepare Model for ML
def prepare_model_data(data, col1, col2):
    x_data = data.drop(columns=[col1, col2])
    y_data = pd.DataFrame(data[col2])
    eval_data = pd.DataFrame(data[[col1, col2]])
    return x_data, y_data, eval_data


## Classifier Functions

In [68]:
#####Classifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVC
def train_classifier_ts(x_data, y_data, eval_data, n_splits, max_train_size):
    merged_data_list = []
    tscv = TimeSeriesSplit(n_splits=n_splits, max_train_size=max_train_size)
    for train_index, test_index in tscv.split(x_data):
        X_train, X_test = x_data.iloc[train_index], x_data.iloc[test_index]
        y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]
        model =  SVC(probability=True)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        predictions = pd.DataFrame(y_pred, index=X_test.index, columns=['pred'])
        merged_data_list.append(pd.merge(predictions, eval_data, left_index=True, right_index=True))
    model_data_train_eval_data = pd.concat(merged_data_list)
    return model_data_train_eval_data, model
def evaluate_classifier_ts(x_data_val, eval_data, model):
    eval_data['pred'] = model.predict(x_data_val)
    model_data_val_eval_data = eval_data
    return model_data_val_eval_data
##### Manual Classifier
def classifier_manual(acutal_value, threshhold_high, threshold_low):
    if acutal_value > threshhold_high:
        return 1
    if acutal_value < threshold_low:
        return 0
    else:
        return None

# Models

## Baseline SVC Classification

In [69]:
strategy = "Baseline_SVC_ts"
pipeline = Pipeline(steps=[
    ('data_copy', FunctionTransformer(create_copy_data, validate=False)),
    ('getReturns', FunctionTransformer(gettransformedOHLC, validate=False, kw_args={'dividend_column': 'Close', 'divisor_column': 'Close', 'dividend_shift' : 0,
                                                                                          'divisor_shift' : 1, 'return_column': 'Returns' })),
    ('getReturns_lag1_df', FunctionTransformer(gettransformedOHLC_df, validate=False, kw_args={'dividend_column': 'Close', 'divisor_column': 'Close', 'dividend_shift' : 1,
                                                                                          'divisor_shift' : 2, 'column_suffix' : '_Returns_lag1' })),
    ('filter_by_symbol', FunctionTransformer(filter_by_symbol, validate=False, kw_args={'symbol': '^SPX'})),
    ('apply_simpleCat', FunctionTransformer(apply_simpleCat, validate=False, kw_args={'column_name': 'Returns'})),
    ('delete_nan', FunctionTransformer(delete_nan, validate=False)),
    ('delete_OHLC', FunctionTransformer(delete_OHLC, validate=False)),
    ('split_data_train_val', FunctionTransformer(split_data_train_val,
     kw_args={'start_date_train': '2015-01-01', 'end_date_train': '2022-12-31', 'start_date_val': '2023-01-01', 'end_date_val': '2023-12-31'}))
])

locals()[f"model_data_train_{strategy}"], locals()[f"model_data_val_{strategy}"] = pipeline.fit_transform(df_Ticker)
locals()[f"x_data_{strategy}"], locals()[f"y_data_{strategy}"], locals()[f"eval_data_{strategy}"] = prepare_model_data(locals()[f"model_data_train_{strategy}"], 'Returns', 'ReturnsCat')
locals()[f"model_data_train_eval_data_{strategy}"], locals()[f"model_{strategy}"] = train_classifier_ts(locals()[f"x_data_{strategy}"], locals()[f"y_data_{strategy}"],locals()[f"eval_data_{strategy}"], 10, 250)
locals()[f"Evaluation_report_train_{strategy}"] = Evaluation_generation(locals()[f"model_data_train_eval_data_{strategy}"], 'pred', 'ReturnsCat', 'Returns')

locals()[f"x_data_val_{strategy}"], locals()[f"y_data_val_{strategy}"], locals()[f"eval_data_val_{strategy}"] = prepare_model_data(locals()[f"model_data_val_{strategy}"], 'Returns', 'ReturnsCat')
locals()[f"model_data_val_eval_data_{strategy}"] = evaluate_classifier_ts(locals()[f"x_data_val_{strategy}"], locals()[f"eval_data_val_{strategy}"], locals()[f"model_{strategy}"])
locals()[f"Evaluation_report_val_{strategy}"] = Evaluation_generation(locals()[f"model_data_val_eval_data_{strategy}"], 'pred', 'ReturnsCat', 'Returns')

### Train Data Evaluation (2015-2022)

The Baseline

In [76]:
print(locals()[f"Evaluation_report_train_{strategy}"] )

                       Metric         Value
0                    Overview      Overview
1                    Accuracy         0.504
2                   Precision      0.538462
3                         NPV      0.432099
4              Return pct lin      13.55357
5         Trade Profit Factor      0.182774
6               Average_Trade      0.054214
7          Average Trade Long      0.106823
8         Average Trade Short     -0.055549
9                Total Trades           250
10      Percent in the Market           1.0
11                Long Trades   Long Trades
12         Total Winners Long            91
13                  Precision      0.538462
14                 Long Ratio         0.548
15             Return pc long     18.053046
16         Profit Factor Long      0.383445
17         Average Trade Long      0.106823
18          Avergage Win Long       0.71576
19          Average Loss Long     -0.603604
20           Largest Win Long      2.284078
21          Total Losers Long   

### Evaluation Data Validation Set (2023)

In [71]:
print(locals()[f"Evaluation_report_val_{strategy}"] )

                       Metric         Value
0                    Overview      Overview
1                    Accuracy         0.516
2                   Precision      0.545977
3                         NPV      0.447368
4              Return pct lin      8.741347
5         Trade Profit Factor      0.114175
6               Average_Trade      0.034965
7          Average Trade Long      0.089925
8         Average Trade Short     -0.090863
9                Total Trades           250
10      Percent in the Market           1.0
11                Long Trades   Long Trades
12         Total Winners Long            95
13                  Precision      0.545977
14                 Long Ratio         0.548
15             Return pc long     15.646935
16         Profit Factor Long      0.321521
17         Average Trade Long      0.089925
18          Avergage Win Long      0.676972
19          Average Loss Long     -0.616017
20           Largest Win Long      2.284078
21          Total Losers Long   

##  Feature Selection

In [72]:
strategy = "Feature_Selection_kbest"
pipeline = Pipeline(steps=[
    ('data_copy', FunctionTransformer(create_copy_data, validate=False)),
    ('getReturns', FunctionTransformer(gettransformedOHLC, validate=False, kw_args={'dividend_column': 'Close', 'divisor_column': 'Close', 'dividend_shift' : 0,
                                                                                          'divisor_shift' : 1, 'return_column': 'Returns' })),
    ('getReturns_lag1_df', FunctionTransformer(gettransformedOHLC_df, validate=False, kw_args={'dividend_column': 'Close', 'divisor_column': 'Close', 'dividend_shift' : 1,
                                                                                          'divisor_shift' : 2, 'column_suffix' : '_Returns_lag1' })),
    ('filter_by_symbol', FunctionTransformer(filter_by_symbol, validate=False, kw_args={'symbol': '^SPX'})),
    ('apply_simpleCat', FunctionTransformer(apply_simpleCat, validate=False, kw_args={'column_name': 'Returns'})),
    ('delete_nan', FunctionTransformer(delete_nan, validate=False)),
    ('delete_OHLC', FunctionTransformer(delete_OHLC, validate=False)),
    ('split_data_train_val', FunctionTransformer(split_data_train_val,
     kw_args={'start_date_train': '2015-01-01', 'end_date_train': '2022-12-31', 'start_date_val': '2023-01-01', 'end_date_val': '2023-12-31'}))
])
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif
locals()[f"model_data_train_{strategy}"], locals()[f"model_data_val_{strategy}"] = pipeline.fit_transform(df_Ticker)
locals()[f"x_data_{strategy}"], locals()[f"y_data_{strategy}"], locals()[f"eval_data_{strategy}"] = prepare_model_data(locals()[f"model_data_train_{strategy}"], 'Returns', 'ReturnsCat') 
fs = SelectKBest(score_func=f_classif, k=70)
x_data_selected = fs.fit_transform(locals()[f"x_data_{strategy}"], locals()[f"y_data_{strategy}"])
selected_feature_indices = fs.get_support(indices=True)
feature_names = list(locals()[f"x_data_{strategy}"].columns)
top_features = [feature_names[i] for i in selected_feature_indices]

## Subset_SVC_ts

In [73]:
strategy = "Subset_SVC_ts"
pipeline = Pipeline(steps=[
    ('data_copy', FunctionTransformer(create_copy_data, validate=False)),
    ('getReturns', FunctionTransformer(gettransformedOHLC, validate=False, kw_args={'dividend_column': 'Close', 'divisor_column': 'Close', 'dividend_shift' : 0,
                                                                                          'divisor_shift' : 1, 'return_column': 'Returns' })),
    ('getReturns_lag1_df', FunctionTransformer(gettransformedOHLC_df, validate=False, kw_args={'dividend_column': 'Close', 'divisor_column': 'Close', 'dividend_shift' : 1,
                                                                                          'divisor_shift' : 2, 'column_suffix' : '_Returns_lag1' })),
    ('filter_by_symbol', FunctionTransformer(filter_by_symbol, validate=False, kw_args={'symbol': '^SPX'})),
    ('apply_simpleCat', FunctionTransformer(apply_simpleCat, validate=False, kw_args={'column_name': 'Returns'})),
    ('delete_nan', FunctionTransformer(delete_nan, validate=False)),
    ('delete_OHLC', FunctionTransformer(delete_OHLC, validate=False)),
    ('split_data_train_val', FunctionTransformer(split_data_train_val,
     kw_args={'start_date_train': '2015-01-01', 'end_date_train': '2022-12-31', 'start_date_val': '2023-01-01', 'end_date_val': '2023-12-31'}))
])

locals()[f"model_data_train_{strategy}"], locals()[f"model_data_val_{strategy}"] = pipeline.fit_transform(df_Ticker)
locals()[f"x_data_{strategy}"], locals()[f"y_data_{strategy}"], locals()[f"eval_data_{strategy}"] = prepare_model_data(locals()[f"model_data_train_{strategy}"], 'Returns', 'ReturnsCat')
locals()[f"x_data_{strategy}"] = locals()[f"x_data_{strategy}"][top_features]
locals()[f"model_data_train_eval_data_{strategy}"], locals()[f"model_{strategy}"] = train_classifier_ts(locals()[f"x_data_{strategy}"], locals()[f"y_data_{strategy}"],locals()[f"eval_data_{strategy}"], 10, 250)
locals()[f"Evaluation_report_train_{strategy}"] = Evaluation_generation(locals()[f"model_data_train_eval_data_{strategy}"], 'pred', 'ReturnsCat', 'Returns')

locals()[f"x_data_val_{strategy}"], locals()[f"y_data_val_{strategy}"], locals()[f"eval_data_val_{strategy}"] = prepare_model_data(locals()[f"model_data_val_{strategy}"], 'Returns', 'ReturnsCat')
locals()[f"x_data_val_{strategy}"] = locals()[f"x_data_val_{strategy}"][top_features]
locals()[f"model_data_val_eval_data_{strategy}"] = evaluate_classifier_ts(locals()[f"x_data_val_{strategy}"], locals()[f"eval_data_val_{strategy}"], locals()[f"model_{strategy}"])
locals()[f"Evaluation_report_val_{strategy}"] = Evaluation_generation(locals()[f"model_data_val_eval_data_{strategy}"], 'pred', 'ReturnsCat', 'Returns')

### Train Data Evaluation (2015-2022)

In [74]:
print(locals()[f"Evaluation_report_train_{strategy}"] )

                       Metric         Value
0                    Overview      Overview
1                    Accuracy      0.521978
2                   Precision       0.53918
3                         NPV        0.4662
4              Return pct lin     43.537207
5         Trade Profit Factor      0.064481
6               Average_Trade      0.023922
7          Average Trade Long      0.043961
8         Average Trade Short     -0.041056
9                Total Trades          1820
10      Percent in the Market           1.0
11                Long Trades   Long Trades
12         Total Winners Long           750
13                  Precision       0.53918
14                 Long Ratio      0.537912
15             Return pc long     61.150407
16         Profit Factor Long       0.11516
17         Average Trade Long      0.043961
18          Avergage Win Long      0.789537
19          Average Loss Long     -0.828396
20           Largest Win Long      9.382774
21          Total Losers Long   

### Evaluation Data Validation Set (2023)

In [75]:
print(locals()[f"Evaluation_report_val_{strategy}"] )

                       Metric         Value
0                    Overview      Overview
1                    Accuracy         0.504
2                   Precision      0.538462
3                         NPV      0.432099
4              Return pct lin      13.55357
5         Trade Profit Factor      0.182774
6               Average_Trade      0.054214
7          Average Trade Long      0.106823
8         Average Trade Short     -0.055549
9                Total Trades           250
10      Percent in the Market           1.0
11                Long Trades   Long Trades
12         Total Winners Long            91
13                  Precision      0.538462
14                 Long Ratio         0.548
15             Return pc long     18.053046
16         Profit Factor Long      0.383445
17         Average Trade Long      0.106823
18          Avergage Win Long       0.71576
19          Average Loss Long     -0.603604
20           Largest Win Long      2.284078
21          Total Losers Long   